In [7]:
import time
import pandas as pd
import time
import pickle
from selenium import webdriver
import os

In [9]:
os.chdir('C:/Users/jaemini_man/Desktop/GitHub/whoscored_crawling')

In [11]:
os.getcwd()

'C:\\Users\\jaemini_man\\Desktop\\GitHub\\whoscored_crawling'

In [14]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https:/1xbet.whoscored.com/')
driver.close()

In [17]:
def crawling_league_teams(team_id, api_delay_term=5):

    """
    cawling league team_id and team name datas
    
    Args :
        team_id : one of you want league team_id & parameter data type int or str
    
    return :
        crawling league team_id, team_name datas belong team_id parameter
        return pandas dataframe columns=team_id, team_name
    
    """
    
    # connect webdriver
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.Chrome('C:/Users/Infosci_center/Desktop/soccer/chromedriver')
    driver.get(url)

    
    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # make pandas dataframe
    team_df = pd.DataFrame(columns=["team_id","team_name"])
    
    # get team datas
    teams = driver.find_elements_by_css_selector("#teams option")
    for team in teams:
        team_name = team.text
        team_id = team.get_attribute("value").split("/")[2]
        team_df.loc[len(team_df)] = {"team_id":team_id, "team_name":team_name }
        
    # close webdriver
    driver.close()
    
    return replace_pd(team_df)

In [18]:
PL = crawling_league_teams(26)

In [19]:
PL

,team_id,team_name
0,13,Arsenal
1,24,Aston Villa
2,211,Brighton
3,184,Burnley
4,15,Chelsea
5,162,Crystal Palace
6,31,Everton
7,170,Fulham
8,19,Leeds
9,14,Leicester


In [83]:
def league_table(URL, api_delay_term=5):
    """
    cwaling league table(seasonal statistics) 
    
    Args : 
        URL : league table URL
    
    Output :
        leauge table(dataframe)
    
 
    """
    url = str(URL)
    driver = webdriver.Chrome('C:/Users/Infosci_center/Desktop/soccer/chromedriver')
    driver.get(url)
    
    time.sleep(api_delay_term)
    
    league_table_df = pd.DataFrame(columns=[
        "team_number", "team_name", "P", "W", "D", "L", "GF", "GA", "GD", "Pts"])
    elements = driver.find_elements_by_css_selector("#standings-17590-content tr")
    
    for element in elements:
        league_table_dict = { 
            "team_number": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].get_attribute("href").split("/")[4], 
            "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].text,     
            "P": element.find_elements_by_css_selector("td")[1].text,
            "W": element.find_elements_by_css_selector("td")[2].text, 
            "D": element.find_elements_by_css_selector("td")[3].text,
            "L": element.find_elements_by_css_selector("td")[4].text, 
            "GF": element.find_elements_by_css_selector("td")[5].text, 
            "GA": element.find_elements_by_css_selector("td")[6].text,
            "GD": element.find_elements_by_css_selector("td")[7].text,
            "Pts": element.find_elements_by_css_selector("td")[8].text,
        }
        league_table_df.loc[len(league_table_df)] = league_table_dict
    
    # close webdriver
    driver.close()
    
    return league_table_df

In [15]:
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/Stages/17590/Show/England-Premier-League-2019-2020' 

In [85]:
PL1920 = league_table(URL)

In [86]:
PL1920

,team_number,team_name,P,W,D,L,GF,GA,GD,Pts
0,26,Liverpool,38,32,3,3,85,33,+52,99
1,167,Manchester City,38,26,3,9,102,35,+67,81
2,32,Manchester United,38,18,12,8,66,36,+30,66
3,15,Chelsea,38,20,6,12,69,54,+15,66
4,14,Leicester,38,18,8,12,67,41,+26,62
5,30,Tottenham,38,16,11,11,61,47,+14,59
6,161,Wolverhampton Wanderers,38,15,14,9,51,40,+11,59
7,13,Arsenal,38,14,14,10,56,48,+8,56
8,163,Sheffield United,38,14,12,12,39,39,0,54
9,184,Burnley,38,15,9,14,43,50,-7,54


In [18]:
def league_table_added(URL, api_delay_term=3):
    """
    crawling league table with additional features
    ex) shot per game, Tackles per game ... etc.
    
    Args : 
        URL : league table URL
        
    Output : 
        league table (data.frame)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(api_delay_term)
    
    league_table_df = pd.DataFrame(columns=[
        "team_name", "P", "W", "D", "L", "GF", "GA", "GD", "Pts"])
    elements = driver.find_elements_by_class_name('standings')[0].find_elements_by_css_selector("tr")
    
    for element in elements:
        league_table_dict = { 
            "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].text,     
            "P": element.find_elements_by_css_selector("td")[1].text,
            "W": element.find_elements_by_css_selector("td")[2].text, 
            "D": element.find_elements_by_css_selector("td")[3].text,
            "L": element.find_elements_by_css_selector("td")[4].text, 
            "GF": element.find_elements_by_css_selector("td")[5].text, 
            "GA": element.find_elements_by_css_selector("td")[6].text,
            "GD": element.find_elements_by_css_selector("td")[7].text,
            "Pts": element.find_elements_by_css_selector("td")[8].text,
        }
        league_table_df.loc[len(league_table_df)] = league_table_dict
    
    time.sleep(api_delay_term)
    
    starter = driver.find_elements_by_id("sub-navigation")
    starter = starter[0].find_elements_by_css_selector("li")[2]
    starter.click()
    
    
    team_stat_df1 = pd.DataFrame(columns=[
        "team_name", "Goals", "Shots pg", "Yellow", "Red", "Poss%", "Pass%", 
        "A_Won", "Rating"
    ])
    elements = driver.find_elements_by_id("top-team-stats-summary-content")
    elements = elements[0].find_elements_by_css_selector("tr")
    
    for element in elements:
        team_table_dict1 = { 
            "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].text.split('. ')[1],     
            "Goals": element.find_elements_by_css_selector("td")[1].text,
            "Shots pg": element.find_elements_by_css_selector("td")[2].text, 
            "Yellow": element.find_elements_by_css_selector("td")[3].find_elements_by_css_selector("span")[0].text, 
            "Red": element.find_elements_by_css_selector("td")[3].find_elements_by_css_selector("span")[1].text, 
            "Poss%": element.find_elements_by_css_selector("td")[4].text, 
            "Pass%": element.find_elements_by_css_selector("td")[5].text,
            "A_Won": element.find_elements_by_css_selector("td")[6].text,
            "Rating": element.find_elements_by_css_selector("td")[7].text,
        }
        team_stat_df1.loc[len(team_stat_df1)] = team_table_dict1
    
    element = driver.find_element_by_css_selector("a[href='#stage-team-stats-defensive']")
    element.click()
    
    time.sleep(api_delay_term)
    
    
    team_stat_df2 = pd.DataFrame(columns=[
        "team_name", "Shoted pg", "Tackles pg", "Intercept pg", "Fouls pg", "Offsides pg"
    ])
    elements = driver.find_elements_by_id("statistics-team-table-defensive")
    elements = elements[0].find_elements_by_id("top-team-stats-summary-content")
    elements = elements[0].find_elements_by_css_selector("tr")
    
    for element in elements:
        team_table_dict2 = { 
            "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].text.split('. ')[1],     
            'Shoted pg': element.find_elements_by_css_selector("td")[1].text,
            'Tackles pg': element.find_elements_by_css_selector("td")[2].text,
            'Intercept pg': element.find_elements_by_css_selector("td")[3].text, 
            'Fouls pg': element.find_elements_by_css_selector("td")[4].text, 
            'Offsides pg': element.find_elements_by_css_selector("td")[5].text, 
        }
        team_stat_df2.loc[len(team_stat_df2)] = team_table_dict2
    
    element = driver.find_element_by_css_selector("a[href='#stage-team-stats-offensive']")
    element.click()
    
    time.sleep(api_delay_term)
    
    team_stat_df3 = pd.DataFrame(columns=[
        "team_name", "Shots OT pg", "Dribbles pg", "Fouled pg"
    ])
    elements = driver.find_elements_by_id("statistics-team-table-offensive")
    elements = elements[0].find_elements_by_id("top-team-stats-summary-content")
    elements = elements[0].find_elements_by_css_selector("tr")
    
    for element in elements: 
        team_table_dict3 = { 
            "team_name": element.find_elements_by_css_selector("td")[0].find_elements_by_css_selector("a")[0].text.split('. ')[1],     
            'Shots OT pg': element.find_elements_by_css_selector("td")[2].text,
            'Dribbles pg': element.find_elements_by_css_selector("td")[3].text, 
            'Fouled pg': element.find_elements_by_css_selector("td")[4].text, 
        }
        team_stat_df3.loc[len(team_stat_df3)] = team_table_dict3
        
        
    team_stat_df = pd.merge(league_table_df, team_stat_df1, how='left', on='team_name')
    team_stat_df = pd.merge(team_stat_df, team_stat_df2, how='left', on='team_name')
    team_stat_df = pd.merge(team_stat_df, team_stat_df3, how='left', on='team_name')
    
    # close webdriver
    driver.close()
    
    return team_stat_df

In [19]:
df = league_table_added(URL)

In [22]:
df.head()

,team_name,P,W,D,L,GF,GA,GD,Pts,Goals,...,A_Won,Rating,Shoted pg,Tackles pg,Intercept pg,Fouls pg,Offsides pg,Shots OT pg,Dribbles pg,Fouled pg
0,Liverpool,38,32,3,3,85,33,+52,99,85,...,17.6,6.94,9,14.5,9.3,8.7,1.4,6.1,10.2,7.7
1,Manchester City,38,26,3,9,102,35,+67,81,102,...,13.6,7.05,7.4,13.5,9.3,9.5,1.8,7,12.8,7.8
2,Manchester United,38,18,12,8,66,36,+30,66,66,...,15,6.83,10.3,15.3,9.9,11.1,1.5,5.7,11.7,11.2
3,Chelsea,38,20,6,12,69,54,+15,66,69,...,18.7,6.86,8.5,16.8,12.1,10.2,1.7,5.9,12.1,10.1
4,Leicester,38,18,8,12,67,41,+26,62,67,...,17.8,6.87,9.9,19.5,11.1,11,1.7,5,9.9,11.7


In [24]:
#df.to_csv('PL1920_league_table.csv')